In [1]:
# Bibliotecas padrão
import os
import io
import sys
import base64
import sqlite3
import json
import logging
from datetime import datetime
from pathlib import Path
from typing import Dict, Any, Optional, Tuple

# Bibliotecas de terceiros
import numpy as np
import torch
from flask import Flask, request, jsonify
from PIL import Image

# Imports para modelos de ML
from transformers import AutoProcessor, AutoModelForCausalLM
import supervision as sv
from datumaro import Bbox, Dataset, DatasetItem

# Configuração de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('oil_detection_server.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Configuração de caminhos para GroundingDINO
HOME = os.getcwd()
CONFIG_PATH = os.path.join(HOME, "GroundingDINO", "groundingdino", "config", "GroundingDINO_SwinT_OGC.py")
%cd {HOME}/GroundingDINO
WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "GroundingDINO\\weights", WEIGHTS_NAME)
from groundingdino.util.inference import load_model, load_image, predict, annotate
%cd {HOME}

c:\Users\Usuario\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\Usuario\Documents\server-defell\GroundingDINO
c:\Users\Usuario\Documents\server-defell


In [2]:
class DatabaseManager:
    """Gerenciador do banco de dados SQLite com estrutura normalizada por empresa"""
    
    def __init__(self, base_path: str = "Overseas"):
        self.base_path = Path(base_path)
        self.base_path.mkdir(exist_ok=True)
        self._databases = {}  # Cache de conexões por empresa
    
    def get_db_path(self, empresa_id: str) -> Path:
        """Retorna o caminho do banco de dados específico da empresa"""
        empresa_dir = self.base_path / empresa_id
        empresa_dir.mkdir(exist_ok=True)
        return empresa_dir / f"{empresa_id}_detection.db"
    
    def init_database(self, empresa_id: str):
        """Inicializa o banco de dados específico da empresa com as tabelas normalizadas"""
        db_path = self.get_db_path(empresa_id)
        conn = sqlite3.connect(str(db_path))
        cursor = conn.cursor()
        
        # Tabela de empresa (uma única empresa por banco)
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS empresa (
                empresa_id TEXT PRIMARY KEY,
                empresa_nome TEXT NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        # Tabela de coletores da empresa
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS coletores (
                coletor_id TEXT PRIMARY KEY,
                coletor_descricao TEXT,
                coletor_localizacao TEXT,
                status TEXT DEFAULT 'ativo',
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        # Tabela principal de detecções
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS deteccoes (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                coletor_id TEXT NOT NULL,
                timestamp_coleta TIMESTAMP NOT NULL,
                timestamp_processamento TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                modelo_usado TEXT NOT NULL,
                imagem_original_path TEXT NOT NULL,
                imagem_processada_path TEXT,
                confidence_yolo REAL,
                objects_detected_yolo INTEGER DEFAULT 0,
                FOREIGN KEY (coletor_id) REFERENCES coletores (coletor_id)
            )
        ''')
        
        # Tabela para captions do Florence-2 (normalizada)
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS florence_captions (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                deteccao_id INTEGER NOT NULL,
                caption_type TEXT NOT NULL,
                caption_text TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (deteccao_id) REFERENCES deteccoes (id)
            )
        ''')
        
        # Tabela para regiões densas do Florence-2
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS florence_dense_regions (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                deteccao_id INTEGER NOT NULL,
                region_text TEXT,
                bbox_x1 REAL,
                bbox_y1 REAL,
                bbox_x2 REAL,
                bbox_y2 REAL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (deteccao_id) REFERENCES deteccoes (id)
            )
        ''')
        
        # Tabela para objetos detectados pelo Florence-2
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS florence_objects (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                deteccao_id INTEGER NOT NULL,
                object_label TEXT,
                bbox_x1 REAL,
                bbox_y1 REAL,
                bbox_x2 REAL,
                bbox_y2 REAL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (deteccao_id) REFERENCES deteccoes (id)
            )
        ''')
        
        # Tabela principal para resultados do GroundingDINO
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS grounding_dino_results (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                deteccao_id INTEGER NOT NULL,
                oil_spill_detected BOOLEAN NOT NULL,
                max_confidence REAL,
                total_detections INTEGER DEFAULT 0,
                image_width INTEGER,
                image_height INTEGER,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (deteccao_id) REFERENCES deteccoes (id)
            )
        ''')
        
        # Tabela para detecções individuais do GroundingDINO
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS grounding_detections (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                grounding_result_id INTEGER NOT NULL,
                phrase TEXT,
                confidence REAL,
                bbox_x1 REAL,
                bbox_y1 REAL,
                bbox_x2 REAL,
                bbox_y2 REAL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (grounding_result_id) REFERENCES grounding_dino_results (id)
            )
        ''')
        
        # Criar índices para melhor performance
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_deteccoes_coletor ON deteccoes(coletor_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_deteccoes_timestamp ON deteccoes(timestamp_coleta)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_deteccoes_modelo ON deteccoes(modelo_usado)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_florence_captions_deteccao ON florence_captions(deteccao_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_florence_captions_type ON florence_captions(caption_type)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_florence_regions_deteccao ON florence_dense_regions(deteccao_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_florence_objects_deteccao ON florence_objects(deteccao_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_grounding_results_deteccao ON grounding_dino_results(deteccao_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_grounding_detections_result ON grounding_detections(grounding_result_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_grounding_detections_confidence ON grounding_detections(confidence)')
        
        conn.commit()
        conn.close()
        logger.info(f"Normalized database initialized successfully for empresa: {empresa_id}")
    
    def insert_empresa(self, empresa_id: str, empresa_nome: str):
        """Insere ou atualiza dados da empresa em seu próprio banco"""
        self.init_database(empresa_id)  # Garante que o banco existe
        
        db_path = self.get_db_path(empresa_id)
        conn = sqlite3.connect(str(db_path))
        cursor = conn.cursor()
        
        cursor.execute('''
            INSERT OR REPLACE INTO empresa (empresa_id, empresa_nome, updated_at)
            VALUES (?, ?, CURRENT_TIMESTAMP)
        ''', (empresa_id, empresa_nome))
        
        conn.commit()
        conn.close()
    
    def insert_coletor(self, empresa_id: str, coletor_id: str, descricao: str, localizacao: str):
        """Insere ou atualiza coletor no banco da empresa"""
        db_path = self.get_db_path(empresa_id)
        conn = sqlite3.connect(str(db_path))
        cursor = conn.cursor()
        
        cursor.execute('''
            INSERT OR REPLACE INTO coletores 
            (coletor_id, coletor_descricao, coletor_localizacao, updated_at)
            VALUES (?, ?, ?, CURRENT_TIMESTAMP)
        ''', (coletor_id, descricao, localizacao))
        
        conn.commit()
        conn.close()
    
    def insert_deteccao(self, empresa_id: str, dados_deteccao: Dict) -> int:
        """Insere uma nova detecção no banco da empresa e retorna o ID"""
        db_path = self.get_db_path(empresa_id)
        conn = sqlite3.connect(str(db_path))
        cursor = conn.cursor()
        
        cursor.execute('''
            INSERT INTO deteccoes 
            (coletor_id, timestamp_coleta, modelo_usado, 
             imagem_original_path, imagem_processada_path, confidence_yolo, objects_detected_yolo)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (
            dados_deteccao['coletor_id'],
            dados_deteccao['timestamp_coleta'],
            dados_deteccao['modelo_usado'],
            dados_deteccao['imagem_original_path'],
            dados_deteccao.get('imagem_processada_path'),
            dados_deteccao.get('confidence_yolo'),
            dados_deteccao.get('objects_detected_yolo', 0)
        ))
        
        deteccao_id = cursor.lastrowid
        conn.commit()
        conn.close()
        return deteccao_id
    
    def insert_florence_result(self, empresa_id: str, deteccao_id: int, florence_data: Dict):
        """Insere resultado do Florence-2 de forma normalizada no banco da empresa"""
        db_path = self.get_db_path(empresa_id)
        conn = sqlite3.connect(str(db_path))
        cursor = conn.cursor()
        
        # Inserir captions simples
        caption_types = {
            '<CAPTION>': 'caption',
            '<DETAILED_CAPTION>': 'detailed_caption',
            '<MORE_DETAILED_CAPTION>': 'more_detailed_caption'
        }
        
        for key, caption_type in caption_types.items():
            caption_text = florence_data.get(key, '')
            if caption_text:
                cursor.execute('''
                    INSERT INTO florence_captions (deteccao_id, caption_type, caption_text)
                    VALUES (?, ?, ?)
                ''', (deteccao_id, caption_type, caption_text))
        
        # Inserir regiões densas
        dense_regions = florence_data.get('<DENSE_REGION_CAPTION>', {})
        if isinstance(dense_regions, dict):
            labels = dense_regions.get('labels', [])
            bboxes = dense_regions.get('bboxes', [])
            
            for i, label in enumerate(labels):
                bbox = bboxes[i] if i < len(bboxes) else [0, 0, 0, 0]
                cursor.execute('''
                    INSERT INTO florence_dense_regions 
                    (deteccao_id, region_text, bbox_x1, bbox_y1, bbox_x2, bbox_y2)
                    VALUES (?, ?, ?, ?, ?, ?)
                ''', (deteccao_id, label, bbox[0], bbox[1], bbox[2], bbox[3]))
        
        # Inserir objetos detectados
        objects_data = florence_data.get('<OD>', {})
        if isinstance(objects_data, dict):
            labels = objects_data.get('labels', [])
            bboxes = objects_data.get('bboxes', [])
            
            for i, label in enumerate(labels):
                bbox = bboxes[i] if i < len(bboxes) else [0, 0, 0, 0]
                cursor.execute('''
                    INSERT INTO florence_objects 
                    (deteccao_id, object_label, bbox_x1, bbox_y1, bbox_x2, bbox_y2)
                    VALUES (?, ?, ?, ?, ?, ?)
                ''', (deteccao_id, label, bbox[0], bbox[1], bbox[2], bbox[3]))
        
        conn.commit()
        conn.close()
    
    def insert_grounding_result(self, empresa_id: str, deteccao_id: int, grounding_data: Dict):
        """Insere resultado do GroundingDINO de forma normalizada no banco da empresa"""
        db_path = self.get_db_path(empresa_id)
        conn = sqlite3.connect(str(db_path))
        cursor = conn.cursor()
        
        detection_data = grounding_data.get('detection_data', {})
        logits = detection_data.get('logits', [])
        boxes = detection_data.get('boxes', [])
        phrases = detection_data.get('phrases', [])
        image_dims = detection_data.get('image_dimensions', (0, 0))
        
        # Converter numpy arrays para listas Python se necessário
        if isinstance(logits, np.ndarray):
            logits_list = logits.tolist()
            has_detections = logits.size > 0
            max_confidence = float(np.max(logits)) if logits.size > 0 else 0.0
            total_detections = int(logits.size)
        else:
            logits_list = list(logits) if logits else []
            has_detections = len(logits_list) > 0
            max_confidence = float(max(logits_list)) if logits_list else 0.0
            total_detections = len(logits_list)
        
        if isinstance(boxes, np.ndarray):
            boxes_list = boxes.tolist()
        else:
            boxes_list = list(boxes) if boxes else []
        
        if isinstance(phrases, np.ndarray):
            phrases_list = phrases.tolist()
        else:
            phrases_list = list(phrases) if phrases else []
        
        # Inserir resultado principal do GroundingDINO
        cursor.execute('''
            INSERT INTO grounding_dino_results 
            (deteccao_id, oil_spill_detected, max_confidence, total_detections, image_width, image_height)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (
            deteccao_id,
            has_detections,
            max_confidence,
            total_detections,
            image_dims[1],  # width
            image_dims[0]   # height
        ))
        
        grounding_result_id = cursor.lastrowid
        
        # Inserir detecções individuais
        for i, confidence in enumerate(logits_list):
            phrase = phrases_list[i] if i < len(phrases_list) else ''
            bbox = boxes_list[i] if i < len(boxes_list) else [0, 0, 0, 0]
            
            cursor.execute('''
                INSERT INTO grounding_detections 
                (grounding_result_id, phrase, confidence, bbox_x1, bbox_y1, bbox_x2, bbox_y2)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (
                grounding_result_id,
                phrase,
                float(confidence),
                float(bbox[0]),
                float(bbox[1]),
                float(bbox[2]),
                float(bbox[3])
            ))
        
        conn.commit()
        conn.close()
    
    def get_detection_with_details(self, empresa_id: str, deteccao_id: int) -> Dict:
        """Recupera uma detecção completa com todos os detalhes normalizados do banco da empresa"""
        db_path = self.get_db_path(empresa_id)
        conn = sqlite3.connect(str(db_path))
        cursor = conn.cursor()
        
        # Buscar detecção principal com dados da empresa e coletor
        cursor.execute('''
            SELECT d.*, e.empresa_nome, c.coletor_descricao, c.coletor_localizacao
            FROM deteccoes d
            JOIN empresa e ON e.empresa_id = ?
            JOIN coletores c ON d.coletor_id = c.coletor_id
            WHERE d.id = ?
        ''', (empresa_id, deteccao_id))
        
        deteccao = cursor.fetchone()
        if not deteccao:
            conn.close()
            return {}
        
        # Buscar captions do Florence
        cursor.execute('''
            SELECT caption_type, caption_text
            FROM florence_captions
            WHERE deteccao_id = ?
        ''', (deteccao_id,))
        captions = {row[0]: row[1] for row in cursor.fetchall()}
        
        # Buscar regiões densas
        cursor.execute('''
            SELECT region_text, bbox_x1, bbox_y1, bbox_x2, bbox_y2
            FROM florence_dense_regions
            WHERE deteccao_id = ?
        ''', (deteccao_id,))
        dense_regions = [
            {
                'text': row[0],
                'bbox': [row[1], row[2], row[3], row[4]]
            }
            for row in cursor.fetchall()
        ]
        
        # Buscar objetos do Florence
        cursor.execute('''
            SELECT object_label, bbox_x1, bbox_y1, bbox_x2, bbox_y2
            FROM florence_objects
            WHERE deteccao_id = ?
        ''', (deteccao_id,))
        florence_objects = [
            {
                'label': row[0],
                'bbox': [row[1], row[2], row[3], row[4]]
            }
            for row in cursor.fetchall()
        ]
        
        # Buscar resultados do GroundingDINO
        cursor.execute('''
            SELECT oil_spill_detected, max_confidence, total_detections, image_width, image_height
            FROM grounding_dino_results
            WHERE deteccao_id = ?
        ''', (deteccao_id,))
        grounding_main = cursor.fetchone()
        
        # Buscar detecções individuais do GroundingDINO
        cursor.execute('''
            SELECT gd.phrase, gd.confidence, gd.bbox_x1, gd.bbox_y1, gd.bbox_x2, gd.bbox_y2
            FROM grounding_detections gd
            JOIN grounding_dino_results gdr ON gd.grounding_result_id = gdr.id
            WHERE gdr.deteccao_id = ?
            ORDER BY gd.confidence DESC
        ''', (deteccao_id,))
        grounding_detections = [
            {
                'phrase': row[0],
                'confidence': row[1],
                'bbox': [row[2], row[3], row[4], row[5]]
            }
            for row in cursor.fetchall()
        ]
        
        conn.close()
        
        # Montar resultado completo
        return {
            'deteccao': {
                'id': deteccao[0],
                'empresa_id': empresa_id,
                'empresa_nome': deteccao[9],
                'coletor_id': deteccao[1],
                'coletor_descricao': deteccao[10],
                'coletor_localizacao': deteccao[11],
                'timestamp_coleta': deteccao[2],
                'timestamp_processamento': deteccao[3],
                'modelo_usado': deteccao[4],
                'imagem_original_path': deteccao[5],
                'imagem_processada_path': deteccao[6],
                'confidence_yolo': deteccao[7],
                'objects_detected_yolo': deteccao[8]
            },
            'florence': {
                'captions': captions,
                'dense_regions': dense_regions,
                'objects': florence_objects
            },
            'grounding_dino': {
                'oil_spill_detected': grounding_main[0] if grounding_main else False,
                'max_confidence': grounding_main[1] if grounding_main else 0.0,
                'total_detections': grounding_main[2] if grounding_main else 0,
                'image_dimensions': [grounding_main[4], grounding_main[3]] if grounding_main else [0, 0],
                'detections': grounding_detections
            }
        }
    
    def get_empresa_stats(self, empresa_id: str) -> Dict:
        """Retorna estatísticas específicas da empresa"""
        try:
            db_path = self.get_db_path(empresa_id)
            if not db_path.exists():
                return {"error": "Database not found for empresa"}
                
            conn = sqlite3.connect(str(db_path))
            cursor = conn.cursor()
            
            # Estatísticas gerais
            cursor.execute("SELECT COUNT(*) FROM deteccoes")
            total_deteccoes = cursor.fetchone()[0]
            
            cursor.execute("SELECT COUNT(*) FROM grounding_dino_results WHERE oil_spill_detected = 1")
            deteccoes_positivas = cursor.fetchone()[0]
            
            cursor.execute("SELECT COUNT(DISTINCT coletor_id) FROM deteccoes")
            total_coletores = cursor.fetchone()[0]
            
            # Top coletores com mais detecções
            cursor.execute('''
                SELECT c.coletor_id, c.coletor_descricao, COUNT(d.id) as total_deteccoes
                FROM coletores c
                LEFT JOIN deteccoes d ON c.coletor_id = d.coletor_id
                GROUP BY c.coletor_id
                ORDER BY total_deteccoes DESC
                LIMIT 5
            ''')
            top_coletores = cursor.fetchall()
            
            conn.close()
            
            return {
                "empresa_id": empresa_id,
                "total_deteccoes": total_deteccoes,
                "deteccoes_positivas": deteccoes_positivas,
                "taxa_deteccao": (deteccoes_positivas / total_deteccoes * 100) if total_deteccoes > 0 else 0,
                "total_coletores": total_coletores,
                "top_coletores": [
                    {
                        "coletor_id": row[0],
                        "descricao": row[1],
                        "total_deteccoes": row[2]
                    }
                    for row in top_coletores
                ]
            }
            
        except Exception as e:
            logger.error(f"Erro ao obter estatísticas da empresa {empresa_id}: {e}")
            return {"error": str(e)}


class FileManager:
    """Gerenciador de arquivos e diretórios por empresa"""
    
    def __init__(self, base_path: str = "Overseas"):
        self.base_path = Path(base_path)
        self.base_path.mkdir(exist_ok=True)
    
    def create_directory_structure(self, empresa_id: str, coletor_id: str, modelo: str) -> Tuple[Path, Path]:
        """Cria estrutura de diretórios específica da empresa e retorna paths para raw e processed"""
        base_dir = self.base_path / empresa_id / coletor_id / modelo
        raw_dir = base_dir / "raw_image"
        processed_dir = base_dir / "processed"
        
        raw_dir.mkdir(parents=True, exist_ok=True)
        processed_dir.mkdir(parents=True, exist_ok=True)
        
        return raw_dir, processed_dir
    
    def generate_filename(self, empresa_id: str, coletor_id: str, timestamp: str, 
                         modelo: str, objects_count: int, prefix: str = "RAW") -> str:
        """Gera nome do arquivo seguindo o padrão especificado"""
        return f"{prefix}-{empresa_id}-{coletor_id}-{timestamp}-{modelo}-{objects_count}.jpg"
    
    def save_image(self, image: Image.Image, filepath: Path) -> bool:
        """Salva imagem no caminho especificado"""
        try:
            image.save(filepath, "JPEG", quality=95)
            return True
        except Exception as e:
            logger.error(f"Erro ao salvar imagem {filepath}: {e}")
            return False
    
    def get_empresa_directory_info(self, empresa_id: str) -> Dict:
        """Retorna informações sobre os diretórios da empresa"""
        empresa_dir = self.base_path / empresa_id
        if not empresa_dir.exists():
            return {"error": "Empresa directory not found"}
        
        info = {
            "empresa_id": empresa_id,
            "base_path": str(empresa_dir),
            "coletores": []
        }
        
        for coletor_dir in empresa_dir.iterdir():
            if coletor_dir.is_dir():
                coletor_info = {
                    "coletor_id": coletor_dir.name,
                    "path": str(coletor_dir),
                    "modelos": []
                }
                
                for modelo_dir in coletor_dir.iterdir():
                    if modelo_dir.is_dir():
                        raw_count = len(list((modelo_dir / "raw_image").glob("*.jpg"))) if (modelo_dir / "raw_image").exists() else 0
                        processed_count = len(list((modelo_dir / "processed").glob("*.jpg"))) if (modelo_dir / "processed").exists() else 0
                        
                        coletor_info["modelos"].append({
                            "modelo": modelo_dir.name,
                            "raw_images": raw_count,
                            "processed_images": processed_count
                        })
                
                info["coletores"].append(coletor_info)
        
        return info
    
class ModelManager:
    """Gerenciador dos modelos Florence-2 e GroundingDINO"""
    
    def __init__(self):
        self.florence_processor = None
        self.florence_model = None
        self.grounding_model = None
        self._load_models()
    
    def _load_models(self):
        """Carrega os modelos na inicialização"""
        try:
            # Carregar Florence-2
            logger.info("Loading Florence-2 model...")
            from transformers import AutoProcessor, AutoModelForCausalLM
            
            self.florence_processor = AutoProcessor.from_pretrained(
                "microsoft/florence-2-base-ft", trust_remote_code=True
            )
            self.florence_model = AutoModelForCausalLM.from_pretrained(
                "microsoft/florence-2-base-ft", trust_remote_code=True
            )
            logger.info("Florence-2 model loaded successfully")
            
            # Carregar GroundingDINO (comentado por enquanto - você precisa ajustar os imports)
            logger.info("Loading GroundingDINO model...")
            self.grounding_model = load_model(CONFIG_PATH, WEIGHTS_PATH, device='cpu')
            logger.info("GroundingDINO model loaded successfully")
            
        except Exception as e:
            logger.error(f"Erro ao carregar modelos: {e}")
    
    def generate_florence_captions(self, image: Image.Image, max_new_tokens: int = 1024) -> Dict:
        """Gera captions usando Florence-2"""
        if not self.florence_model or not self.florence_processor:
            logger.warning("Florence-2 model not loaded, returning empty results")
            return {}
        
        prompts = [
            "<CAPTION>",
            "<DETAILED_CAPTION>", 
            "<MORE_DETAILED_CAPTION>",
            "<DENSE_REGION_CAPTION>",
            "<OD>",
        ]
        
        results = {}
        for prompt in prompts:
            try:
                inputs = self.florence_processor(text=prompt, images=image, return_tensors="pt")
                generated_ids = self.florence_model.generate(
                    input_ids=inputs["input_ids"],
                    pixel_values=inputs["pixel_values"],
                    max_new_tokens=max_new_tokens
                )
                generated_text = self.florence_processor.batch_decode(
                    generated_ids, skip_special_tokens=False
                )[0]
                parsed_answer = self.florence_processor.post_process_generation(
                    generated_text,
                    task=prompt,
                    image_size=(image.width, image.height)
                )
                results[prompt] = parsed_answer.get(prompt, "")
            except Exception as e:
                logger.error(f"Erro no Florence-2 com prompt {prompt}: {e}")
                results[prompt] = ""
        
        return results
    
    def process_grounding_detection(self, image: Image.Image, 
                              text_prompt: str = "oil spill",
                              box_threshold: float = 0.3,
                              text_threshold: float = 0.2) -> Dict:
        """Processa detecção com GroundingDINO"""
        try:
            if not self.grounding_model:
                logger.warning("GroundingDINO model not loaded, returning empty results")
                return {
                    'annotated_image': np.array(image),
                    'detection_data': {
                        'boxes': np.array([]),
                        'logits': np.array([]),
                        'phrases': [],
                        'image_dimensions': (image.height, image.width)
                    }
                }
            
            # Converter PIL Image para numpy array
            image_array = np.array(image)
            
            # Converter imagem para tensor
            image_tensor = torch.from_numpy(image_array).permute(2, 0, 1).float()
            
            # Processar detecção
            self.grounding_model.eval()
            with torch.no_grad():
                boxes, logits, phrases = predict(
                    model=self.grounding_model,
                    image=image_tensor,
                    caption=text_prompt,
                    box_threshold=box_threshold,
                    text_threshold=text_threshold,
                    device='cpu'
                )
            
            # Converter tensores para numpy
            boxes_np = boxes.cpu().numpy()
            logits_np = logits.cpu().numpy()
            
            # Anotar a imagem
            annotated_frame = annotate(image_array, boxes, logits, phrases)
            
            return {
                'annotated_image': annotated_frame,
                'detection_data': {
                    'boxes': boxes_np,
                    'logits': logits_np,
                    'phrases': phrases,
                    'image_dimensions': (image.height, image.width)
                }
            }
            
        except Exception as e:
            logger.error(f"Erro no GroundingDINO: {e}")
            # Retornar estrutura vazia em caso de erro
            return {
                'annotated_image': np.array(image),
                'detection_data': {
                    'boxes': np.array([]),
                    'logits': np.array([]),
                    'phrases': [],
                    'image_dimensions': (image.height, image.width)
                }
            }


In [3]:
class OilDetectionServer:
    """Servidor principal para detecção de óleo"""
    
    def __init__(self):
        self.app = Flask(__name__)
        self.db_manager = DatabaseManager()
        self.file_manager = FileManager()
        self.model_manager = ModelManager()
        self._setup_routes()
    
    def _setup_routes(self):
        """Configura as rotas do Flask"""
        
        @self.app.route('/oil_detection', methods=['POST'])
        def oil_detection_endpoint():
            return self.process_oil_detection()
        
        @self.app.route('/processar', methods=['POST'])
        def processar():
            return self.legacy_processar()
        
        @self.app.route('/skf_analysis', methods=['POST'])
        def handle_skf_analysis():
            return self.skf_analysis_endpoint()
        
        @self.app.route('/health', methods=['GET'])
        def health_check():
            return jsonify({"status": "healthy", "timestamp": datetime.now().isoformat()})
        
        @self.app.route('/stats', methods=['GET'])
        def get_stats():
            return self.get_detection_stats()
        
        @self.app.route('/stats/<empresa_id>', methods=['GET'])
        def get_empresa_stats(empresa_id):
            return self.get_empresa_stats(empresa_id)
        
        @self.app.route('/detection/<empresa_id>/<int:deteccao_id>', methods=['GET'])
        def get_detection_details(empresa_id, deteccao_id):
            return self.get_detection_details(empresa_id, deteccao_id)
    
    def process_oil_detection(self):
        """Endpoint principal para processamento de detecção de óleo"""
        try:
            data = request.get_json()
            
            # Validar dados obrigatórios
            required_fields = ['imagem_original_base64', 'empresa_info']
            for field in required_fields:
                if field not in data:
                    return jsonify({
                        "status": "error", 
                        "message": f"Campo obrigatório ausente: {field}"
                    }), 400
            
            # Extrair dados da estrutura empresa_info
            empresa_info = data['empresa_info']
            empresa_id = empresa_info['empresa_id']
            empresa_nome = empresa_info['empresa_nome']
            coletor_id = empresa_info['coletor_id']
            coletor_descricao = empresa_info.get('coletor_descricao', '')
            coletor_localizacao = empresa_info.get('localizacao', '')
            modelo_usado = empresa_info.get('modelo_id', 'oil_spill')
            confidence_yolo = data.get('confidence_threshold')
            timestamp_coleta = data.get('timestamp', datetime.now().isoformat())
            
            # Validação e conversão segura da imagem
            try:
                imagem_bytes = base64.b64decode(data['imagem_original_base64'])
                image = Image.open(io.BytesIO(imagem_bytes)).convert("RGB")
                logger.info(f"Image loaded successfully: {image.size}")
            except Exception as e:
                logger.error(f"Erro ao processar imagem base64: {e}")
                return jsonify({
                    "status": "error", 
                    "message": f"Erro ao processar imagem: {str(e)}"
                }), 400
            
            # Inserir/atualizar empresa e coletor no banco específico da empresa
            self.db_manager.insert_empresa(empresa_id, empresa_nome)
            self.db_manager.insert_coletor(empresa_id, coletor_id, coletor_descricao, coletor_localizacao)
            
            # Criar estrutura de diretórios específica da empresa
            raw_dir, processed_dir = self.file_manager.create_directory_structure(
                empresa_id, coletor_id, modelo_usado
            )
            
            # Processar com Florence-2
            logger.info(f"Processing image with Florence-2 for {empresa_id}/{coletor_id}")
            florence_results = self.model_manager.generate_florence_captions(image)
            
            # Processar com GroundingDINO
            logger.info(f"Processing image with GroundingDINO for {empresa_id}/{coletor_id}")
            grounding_results = self.model_manager.process_grounding_detection(image)
            
            # Calcular detecções de forma segura
            detection_data = grounding_results.get('detection_data', {})
            logits = detection_data.get('logits', np.array([]))
            
            # Usar contagem de detecções do payload se existir, senão calcular
            if isinstance(logits, np.ndarray):
                objects_detected = data.get('detections_count', int(logits.size))
            else:
                objects_detected = data.get('detections_count', len(logits) if logits else 0)
            
            # Gerar nomes de arquivo
            timestamp_str = datetime.now().strftime("%Y%m%d%H%M%S%f")[:-3]
            raw_filename = data.get('name_imagem_original', 
                                  self.file_manager.generate_filename(
                                      empresa_id, coletor_id, timestamp_str, modelo_usado, objects_detected, "RAW"
                                  ))
            processed_filename = data.get('name_imagem_processada',
                                        self.file_manager.generate_filename(
                                            empresa_id, coletor_id, timestamp_str, modelo_usado, objects_detected, "PROC"
                                        ))
            
            # Salvar imagens
            raw_path = raw_dir / raw_filename
            processed_path = processed_dir / processed_filename
            
            # Verificar se salvou corretamente
            if not self.file_manager.save_image(image, raw_path):
                logger.warning(f"Failed to save raw image to {raw_path}")
            
            # Salvar imagem processada se existir no payload
            processed_image_saved = False
            if 'imagem_processada_base64' in data:
                try:
                    processed_bytes = base64.b64decode(data['imagem_processada_base64'])
                    processed_image = Image.open(io.BytesIO(processed_bytes)).convert("RGB")
                    processed_image_saved = self.file_manager.save_image(processed_image, processed_path)
                except Exception as e:
                    logger.error(f"Erro ao processar imagem processada: {e}")
            elif 'annotated_image' in grounding_results:
                try:
                    annotated_array = grounding_results['annotated_image']
                    if isinstance(annotated_array, np.ndarray) and annotated_array.size > 0:
                        processed_image = Image.fromarray(annotated_array.astype('uint8'))
                        processed_image_saved = self.file_manager.save_image(processed_image, processed_path)
                except Exception as e:
                    logger.error(f"Erro ao salvar imagem anotada: {e}")
            
            # Preparar dados para inserção no banco
            dados_deteccao = {
                'coletor_id': coletor_id,
                'timestamp_coleta': timestamp_coleta,
                'modelo_usado': modelo_usado,
                'imagem_original_path': str(raw_path),
                'imagem_processada_path': str(processed_path) if processed_image_saved else None,
                'confidence_yolo': confidence_yolo,
                'objects_detected_yolo': objects_detected
            }
            
            # Inserir no banco de dados específico da empresa
            deteccao_id = self.db_manager.insert_deteccao(empresa_id, dados_deteccao)
            self.db_manager.insert_florence_result(empresa_id, deteccao_id, florence_results)
            self.db_manager.insert_grounding_result(empresa_id, deteccao_id, grounding_results)
            
            # Calcular max_confidence de forma segura
            if isinstance(logits, np.ndarray):
                max_confidence = float(np.max(logits)) if logits.size > 0 else 0.0
            else:
                max_confidence = float(max(logits)) if logits else 0.0
            
            # Preparar resposta
            response = {
                "status": "success",
                "deteccao_id": deteccao_id,
                "empresa_id": empresa_id,
                "objects_detected": objects_detected,
                "oil_spill_detected": objects_detected > 0,
                "max_confidence": max_confidence,
                "florence_caption": florence_results.get('<CAPTION>', ''),
                "files_saved": {
                    "raw_image": str(raw_path),
                    "processed_image": str(processed_path) if processed_image_saved else None
                },
                "timestamp_processamento": datetime.now().isoformat()
            }
            
            logger.info(f"Successfully processed detection {deteccao_id} for {empresa_id}/{coletor_id}")
            return jsonify(response)
            
        except Exception as e:
            logger.error(f"Erro no processamento: {e}", exc_info=True)
            return jsonify({
                "status": "error",
                "message": f"Erro interno do servidor: {str(e)}"
            }), 500
        
    def legacy_processar(self):
        """Endpoint legacy para compatibilidade"""
        try:
            dados = request.get_json()
            imagem_b64 = dados.get('imagem_base64')
            nome_imagem = dados.get('nome_imagem', 'imagem_recebida.jpg')
            texto = dados.get('texto', '')
            
            if not imagem_b64:
                return jsonify({"resposta": "Imagem base64 não fornecida"}), 400
            
            # Salva a imagem temporariamente
            with open(nome_imagem, "wb") as f:
                f.write(base64.b64decode(imagem_b64))
            
            tamanho_kb = os.path.getsize(nome_imagem) / 1024
            resposta = f"{nome_imagem}: {tamanho_kb:.1f}kb | Texto recebido: {texto}"
            
            return jsonify({"resposta": resposta})
        except Exception as e:
            logger.error(f"Erro no endpoint legacy: {e}")
            return jsonify({"resposta": f"Erro: {str(e)}"}), 500
    
    def skf_analysis_endpoint(self):
        """Endpoint para análise SKF (Florence-2)"""
        try:
            data = request.json
            
            imagem_base64 = data.get("imagem_base64")
            if not imagem_base64:
                return jsonify({"status": "error", "message": "Imagem base64 não fornecida"}), 400
            
            # Converter base64 para imagem
            imagem_bytes = base64.b64decode(imagem_base64)
            image_stream = io.BytesIO(imagem_bytes)
            image = Image.open(image_stream).convert("RGB")
            
            # Processar com Florence-2
            result = self.model_manager.generate_florence_captions(image)
            
            return jsonify({
                "status": "success",
                "return": result
            })
            
        except Exception as e:
            logger.error(f"Erro no SKF analysis: {e}")
            return jsonify({
                "status": "error",
                "message": f"Erro no servidor: {str(e)}"
            }), 500
    
    def get_detection_stats(self):
        """Retorna estatísticas globais de todas as empresas"""
        try:
            # Buscar todas as empresas no diretório base
            base_path = Path(self.db_manager.base_path)
            if not base_path.exists():
                return jsonify({
                    "status": "success",
                    "stats": {
                        "total_empresas": 0,
                        "total_deteccoes": 0,
                        "deteccoes_positivas": 0,
                        "taxa_deteccao": 0,
                        "empresas": []
                    }
                })
            
            empresas_stats = []
            total_deteccoes = 0
            total_positivas = 0
            
            for empresa_dir in base_path.iterdir():
                if empresa_dir.is_dir():
                    empresa_id = empresa_dir.name
                    stats = self.db_manager.get_empresa_stats(empresa_id)
                    
                    if "error" not in stats:
                        empresas_stats.append(stats)
                        total_deteccoes += stats["total_deteccoes"]
                        total_positivas += stats["deteccoes_positivas"]
            
            return jsonify({
                "status": "success",
                "stats": {
                    "total_empresas": len(empresas_stats),
                    "total_deteccoes": total_deteccoes,
                    "deteccoes_positivas": total_positivas,
                    "taxa_deteccao": (total_positivas / total_deteccoes * 100) if total_deteccoes > 0 else 0,
                    "empresas": empresas_stats
                }
            })
            
        except Exception as e:
            logger.error(f"Erro ao obter estatísticas globais: {e}")
            return jsonify({
                "status": "error",
                "message": str(e)
            }), 500
    
    def get_empresa_stats(self, empresa_id: str):
        """Retorna estatísticas específicas de uma empresa"""
        try:
            stats = self.db_manager.get_empresa_stats(empresa_id)
            
            if "error" in stats:
                return jsonify({
                    "status": "error",
                    "message": stats["error"]
                }), 404
            
            # Adicionar informações de diretório
            dir_info = self.file_manager.get_empresa_directory_info(empresa_id)
            if "error" not in dir_info:
                stats["directory_info"] = dir_info
            
            return jsonify({
                "status": "success",
                "stats": stats
            })
            
        except Exception as e:
            logger.error(f"Erro ao obter estatísticas da empresa {empresa_id}: {e}")
            return jsonify({
                "status": "error",
                "message": str(e)
            }), 500
    
    def get_detection_details(self, empresa_id: str, deteccao_id: int):
        """Retorna detalhes completos de uma detecção específica"""
        try:
            detection_data = self.db_manager.get_detection_with_details(empresa_id, deteccao_id)
            
            if not detection_data:
                return jsonify({
                    "status": "error",
                    "message": "Detecção não encontrada"
                }), 404
            
            return jsonify({
                "status": "success",
                "data": detection_data
            })
            
        except Exception as e:
            logger.error(f"Erro ao obter detalhes da detecção {deteccao_id} da empresa {empresa_id}: {e}")
            return jsonify({
                "status": "error",
                "message": str(e)
            }), 500
    
    def run(self, host="0.0.0.0", port=8080, debug=False):
        """Executa o servidor"""
        logger.info(f"Starting Oil Detection Server on {host}:{port}")
        self.app.run(host=host, port=port, debug=debug)

In [4]:
if __name__ == "__main__":
    
    # Verificar se estamos no processo principal (evitar problemas com reloader)
    if os.environ.get('WERKZEUG_RUN_MAIN') != 'true': print("Initializing Oil Detection Server...")
    
    server = OilDetectionServer()
        
    # Executar com debug desabilitado ou com reloader desabilitado
    debug_mode = "--debug" in sys.argv
    server.run(debug=debug_mode)

2025-06-14 18:49:45,737 - INFO - Loading Florence-2 model...


Initializing Oil Detection Server...


Florence2LanguageForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
2025-06-14 18:50:25,964 - INFO - Florence-2 model loaded successfully
2025-06-14 18:50:25,966 - INFO - Loading GroundingDINO model...


final text_encoder_type: bert-base-uncased


2025-06-14 18:50:40,431 - INFO - GroundingDINO model loaded successfully
2025-06-14 18:50:40,434 - INFO - Starting Oil Detection Server on 0.0.0.0:8080


 * Serving Flask app '__main__'
 * Debug mode: off


2025-06-14 18:50:40,440 - INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://10.2.0.2:8080
2025-06-14 18:50:40,441 - INFO - Press CTRL+C to quit
2025-06-14 18:51:11,350 - INFO - 127.0.0.1 - - [14/Jun/2025 18:51:11] "GET /health HTTP/1.1" 200 -
2025-06-14 18:51:23,576 - INFO - Image loaded successfully: (640, 480)
2025-06-14 18:51:23,752 - INFO - Normalized database initialized successfully for empresa: DOS0724
2025-06-14 18:51:23,773 - INFO - Processing image with Florence-2 for DOS0724/APS005
2025-06-14 18:51:34,129 - INFO - Processing image with GroundingDINO for DOS0724/APS005
2025-06-14 18:51:36,439 - INFO - Successfully processed detection 1 for DOS0724/APS005
2025-06-14 18:51:36,440 - INFO - 127.0.0.1 - - [14/Jun/2025 18:51:36] "POST /oil_detection HTTP/1.1" 200 -
